# Data handling with `data.table`


## Why `R`

* An integrated workflow
* State of the art statistics, data mining, visualisation...
* Transparency (a peer can read the 'recipe')

![R + data.table](https://lh3.googleusercontent.com/bC_VxRnj88vsK3sxRhATGxJXuwMVjLCTpCzo6EErneHkczRq-tOZDPRTel_DEw2E152pfVnH5_Wr05CYxmiiNYb1LOtq51HuYSFo2GiwRkcoCwPUYNIimJJ7TxMbFB_K83-jYc2ynA=w952-h330-no)


## About these notebooks

1. File/make a copy/...
2. (rename)/save
3. Double click on a **cell** to edit
4. SHIFT + ENTER to run the whole cell and move to the next

In [ ]:
## try to write some R code here
a = 1
a + 5 / 3


## Data manipulation, transformation and curation

In the real world, data:

* is rarely in the correct format (collaborators!!),
* may need to be summarised,
* has some mistakes (need quality control),
* can be enriched (merging data from multiple sources), and
* manipulation comes with (implicit) assumptions.

![data cleaning](https://www.kdnuggets.com/wp-content/uploads/data-cleaning-1.jpg)


## Data handling good practices

* **Read-only** input data (stick to [CSV](https://en.wikipedia.org/wiki/Comma-separated_values))
* Script internally **copies the data** 
* Script is a **linear sequence** of operations
* Interactive and critical process 

![https://r4ds.had.co.nz/explore-intro.html](https://d33wubrfki0l68.cloudfront.net/795c039ba2520455d833b4034befc8cf360a70ba/558a5/diagrams/data-science-explore.png)

## Statistical data frames in `R` (`data.frame`)

* One column per variable
* One row per observation
* Generic and consistent format for visualisation and modelling
* But, old and clunky

## `data.table`, an improved `data.frame`

* Efficient syntax
* Performance
* Conceptual improvements (key, joins, ...)
* Fully compatible with `data.frame`
* See also the [tidyverse](https://www.tidyverse.org/)


## `data.table`, a matter of syntax


```R
DT[i, j, by]
```

Means 'Take DT, subset/reorder rows using `i`, then calculate `j`, grouped by `by`'.


# Today's data

## Background story

You are deeply interested in the effect of agricultural practices on insect pest pressure.
Your charismatic advisor has convinced you to set up an ambitious monitoring program to record the **occurrence of a pest insect**. 
In addition, you have decided to score the presence of its most **significant predator**, as well as a **generalist parasitoid**. 
By some miracle, you have managed to acquire a large amount of sticky trap data (well done!).


## Your variables 

* For a whole season, you (and your dedicated field assistant) have placed multiple traps for seven days, and counted the **number** of all three players `N_pest`, `N_pred`, `N_para` (those are three response variables).

* Each week, at a specific date (`date`), four traps will have been put on one of six sites (`site`): `'A'`, `'B'`, `C`, `D`, `E` and `F`. 

* Each site is characterised by one of two practices (`practice`): `orga`, `conv`

The field season has just ended, and you can finally sit at a desk, enter these data, and enjoy the fun part!


## Data structure

You had the common sense of organising your data in a single CSV file `2019-ag_practices_effect_on_pest.csv`, which has a sensible name, (available [here](https://raw.githubusercontent.com/qgeissmann/r_workshops/master/2019-ag_practices_effect_on_pest.csv)). **Click on the data the [link](https://raw.githubusercontent.com/qgeissmann/r_workshops/master/2019-ag_practices_effect_on_pest.csv) and have a look yourself**.

Each row corresponds to a single trap.
The columns contain the three response variables that were recorded on each trap: `N_pest`, `N_pred`, and `N_para`.
In addition, the `date`, `site` and `practice` of each defined as additional columns.




## Import and curation

When putting your hands on some data, you will pretty much always want to do the following:

1. Import the `data.table` 
1. Load the data in using `fread`
1. Check the dimension of the resulting table
1. Check / fix the variable types
1. Look for obvious irregularities 


### Package import

In [ ]:
# On your own machine install data table first:
# install.packages('data.table')
library(data.table)


### Loading data

You can either [download your data](https://raw.githubusercontent.com/qgeissmann/r_workshops/master/2019-ag_practices_effect_on_pest.csv), or let `fread` retrieve it from the web, as in:

In [ ]:
URL <- "https://github.com/qgeissmann/r_workshops/raw/master/2019-ag_practices_effect_on_pest.csv"
dt <- fread(URL, stringsAsFactors = TRUE) # forces string as factors

### Dimensions
You can get dimensions by just printing the table. It also shows the type of each variable:

In [ ]:
dt # same as print(dt)

### Changing column type

`date`, should be a continuous variable -- as opposed to a factor. This will become very relevant when modelling, visualising and processing.
Sometimes you want, for instance, to force variables to be characters instead of numbers etc.

In [ ]:
## Data tables ARE data frames, so you can just use 
dt$date <- as.Date(dt$date)

In [ ]:
## A consise way to summarise variables in R:
str(dt)

## Data summaries

To find more irregularity, we need to 'question' the data. The simplest way is to generate 'aggregates' specific questions. For example,

* What is the **total number of traps for each site**?
* What is the **total number of traps for each date**?
* What is the **total number of traps for each date AND site**?
* What is the **sum of all three insects for each date**?

This can be done using the syntax:

```R
DT[,XXX,by=YYY]
```
where `YYY` are the variables to use as keys (e.g. site, date,...) and `XXX`, the operation to perform.


In [ ]:
# .N is a special variable returning the number
dt[, .N, by='site']

In [ ]:
## YOUR TURN
## How many observations per date?



In [ ]:
## Instead of .N, we can do some arbitrary operations on columns:
dt[, sum(N_pred), by='site']

In [ ]:
## The total of all insect FOR EACH DATE
dt[, sum(N_pred + N_para + N_pest), by=date]

In [ ]:
## YOUR TURN
## Ask your own question here

# dt[,,by=]

## Filtering and selection (rows)

Sometimes you want to subset your data -- i.e. select only some observations.
This can be done with the `DT[III,,]` syntax, where `III` is a condition.

In [ ]:
## For example, to show only read from site A
dt[site == 'A']

In [ ]:
## To overwrite dt, we need to use <- (or =)
dt <- dt[date >= "2019-04-05"] ## note, this condition works because date IS continuous
str(dt)

In [ ]:
## YOUR TURN (a tricky one!)
## What is the average (mean) number of pests, FOR EACH date, 
## ONLY for observations that have AT LEAST 10 predators


## Alteration and addition of variables

Most likely you will also want to make new variables, or redefine them.
For instance, we changed date earlier with the base `R` syntax 

```R
DF$VARIABLE <- ZZZ(DF$VARIABLE)
```

where `ZZZ` is a function. 

In `data.table`, there is a much more elegant and efficient syntax: 

```R
DT[, VARIABLE := ZZZ(VARIABLE)]
```

In [ ]:
# To make a new variable that is the log10(N_pest +1):
dt[, log_N_pest := log10(N_pest +1)]
dt

In [ ]:
## You can also make operations between variables
dt[, sum_all_insects := N_pest + N_para + N_pred]
str(dt)

In [ ]:
## Remove variables
dt[, sum_all_insects := NULL]
dt[, log_N_pest:= NULL]

str(dt)

In [ ]:
## YOUR TURN (easy)
## compute the ratio of predator over pest (what are the possible issues)
# dt[, pred_over_pest := ....]

## YOUR TURN (harder)
## Compute the cumulative sum  of all pest along the season, FOR EACH site
## Which critical assuption are you making?!!

# dt[, cum_N_pest := cumsum(...),]


## Joins

This is where `data.table` shines the most 🌞!
It is sometimes inconvenient to keep "metadata" in the same data table.
For instance, you may have **another table** that contains a description of each site in terms of location, soil pH, elevation etc. If you want to include any of these "metavariables" in a model/plot, you will need to add it to your data. In other words, you want to find all the rows for a given site and append them with the corresponding data from another table. This operation is formally called a [**join**](https://en.wikipedia.org/wiki/Relational_algebra#Joins_and_join-like_operators). 

To keep things simple, let's forge such a 'metadata' table by hand (in practice, you would have another csv, but it is just an exemple and I was being lazy): 

In [ ]:
meta_dt <- data.table(site=      c('A', 'B', 'C', 'D', 'E', 'F'),
                      elevation= c(100, 20,  300, 70,  110, 50),
                      pH =       c(7.5, 5.8, 8.2, 6.2, 7.1, 6.5)
                     )
meta_dt

Now the magic formula is `MDT[DT, on = KEY]`🧙‍,
where `MDT` is the metadata, and `KEY`, is the name of the colum(S) to 'map'.
In practice:

In [ ]:
enriched_dt <- meta_dt[dt, on='site']
enriched_dt

Joins can also be applied ON continuous variables, for example on `date`.
You could, for example, get some regional weather data as another table, ensure it has a column named `date`, and join it like so: `weather_dt[dt, on='date']`. 

This will only match the dates in `dt` that are exactly the same as in `weather_dt`. 
You can apply a "rolling join" to allow a match with close dates instead, but we won't do that today.

## Reshaping data

In our data, we have three response variables `N_pest`, `N_pred` and `N_para`.
However, we are recording the same thing, a number of individuals, on tree distinct species. 
In some cases (modelling and visualisation), we would like to encode the count as a unique response variable,
and include a new variable, e.g. `species` as a covariate. The original table is encoded in a **wide** format, while we want a **long** format. Importantly, **both tables hold the exact same information**. The `melt` function can be used to melt a wide to a long table. For the reverse operation, long to wide, use `dcast`.



In [ ]:
dt_long <- melt(dt,
                measure.vars = patterns('N_*'),
                value.name='N',
                variable.name ='species')
dt_long[, species := gsub("^N_", "", species)]
dt_long


##  Visualisation (teaser)

The next step would be to visualise the data, that should happen as a quality control before any modelling / statistics.
For instance, we can show the temporal trend of the count, for each species (as a different colour) and each site/practice (as a subplot) using some `ggplot` whitchcraft:


In [ ]:
library(ggplot2)
options(repr.plot.width=12, repr.plot.height=9)
ggplot(dt_long, aes(date, N, colour=species)) + 
            geom_point(alpha=.3) + 
            geom_smooth(method='loess') + 
            facet_wrap( ~ practice * site)

## Resources

* [Official tutorial](https://cran.r-project.org/web/packages/data.table/vignettes/datatable-intro.html)
* [Data table cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/datatable_Cheat_Sheet_R.pdf)
* Rolling joins [tutorials](https://www.r-bloggers.com/understanding-data-table-rolling-joins/)
* [Video description](https://www.youtube.com/watch?v=MvH1eTdsekA) of `data.table` by its creator